In [10]:
# I do not want to manually rerun every single folder
import os
from eval_manager import run_eval_on_dir

In [11]:
for root, dirs, files in os.walk("experiments"):
    for dir in dirs:
        date = dir[4:6]
        dates = ['19','20','21','22','23']
        if dir[:3] == "jan" and (date in dates):
            try:
                run_eval_on_dir(os.path.join(root,dir))
                print("Ran ", dir)
            except:
                print("Could not run ", dir)

2024-04-25 15:45:19,521 - EvalManager - DEBUG - running evaluator on experiments/jan_22_eps0_recipes_100_users_random/noise_0.25_selection_random_mnli_temp_0
2024-04-25 15:45:19,540 - EvalManager - DEBUG - running evaluator on experiments/jan_22_eps0_recipes_100_users_random/noise_0.5_selection_random_mnli_temp_0
2024-04-25 15:45:19,566 - EvalManager - DEBUG - running evaluator on experiments/jan_22_eps0_recipes_100_users_random/noise_0_selection_random_mnli_temp_0
Ran  jan_22_eps0_recipes_100_users_random
2024-04-25 15:45:19,781 - EvalManager - DEBUG - running evaluator on experiments/jan_22_restaurants_no_history/noise_0_selection_thompson_mnli_temp_1
2024-04-25 15:45:19,802 - EvalManager - DEBUG - running evaluator on experiments/jan_22_restaurants_no_history/noise_0_selection_ucb_mnli_temp_10
Ran  jan_22_restaurants_no_history
2024-04-25 15:45:20,152 - EvalManager - DEBUG - running evaluator on experiments/jan_21_movies_100_users_noise0.25_random/noise_0.25_selection_random_mnli_te

### Move aggregated results from experiments folder to corresponding organized_results folder

In [12]:
# Load mapping from exp folders to organized results folders
import json
with open("eval_map.json","r") as map_file:
    folder_map = json.load(map_file)

In [13]:
import shutil
for key,val in folder_map.items():
    try:
        shutil.copyfile(os.path.join(key, "aggregated_results.csv"), os.path.join('organized_results', val, "aggregated_results.csv"))
    except:
        print(f"Could not copy from {key} to {val}")

## Copy into Best file

In [5]:
import pandas as pd

In [17]:

noise_levels = [0, 0.25, 0.5]
methods = ['mono', 'er', 'greedy', 'random', 'thompson','ucb']
pos_sets = [[0,1,1,0,0,1],
           [1,2,2,0,0,1],
           [2,0,1,0,1,1]
           ] # Relative position of row in own table (this indicates which mnli temp to use)
datasets = ['movies','recipes','restaurants']

temp_df = pd.read_csv(f"organized_results/movies/er/noise0/aggregated_results.csv")
cols = temp_df.columns

for noise_idx, noise_level in enumerate(noise_levels):
    pos_set = pos_sets[noise_idx]

    for dataset in datasets:
        new_df_rows = []
        for method_idx, method in enumerate(methods):
            if method == "mono":
                df = pd.read_csv(f"organized_results/{dataset}/{method}/aggregated_results.csv")
            else:
                df = pd.read_csv(f"organized_results/{dataset}/{method}/noise{noise_level}/aggregated_results.csv")
            new_df_rows.append(df.iloc[pos_set[method_idx], :])

        new_df = pd.DataFrame(new_df_rows, columns=cols)
        new_df.to_csv(f"organized_results/{dataset}/best/noise{noise_level}/aggregated_results.csv", index=False)

